In [ ]:
```json
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# MediaPipe Beyond Hand Tracking - Eksplorasi Komprehensif\n",
    "**Tugas Besar Pengolahan Citra Digital**  \n",
    "**Tim:** Rindi Indriani, Acit, Dian  \n",
    "**Tanggal:** 14 Juni 2025  \n",
    "**PIC Notebook:** Rindi Indriani\n",
    "\n",
    "## 🎯 Tujuan Eksplorasi\n",
    "Berdasarkan aplikasi baseline yang menggunakan **HEAD GESTURE CONTROL** (Face Mesh untuk deteksi tilt kepala), eksplorasi ini bertujuan:\n",
    "\n",
    "1. Menganalisis performa baseline **Head Gesture Control** (Face Mesh current system)\n",
    "2. Mengeksplorasi **MediaPipe Pose** untuk full body tracking\n",
    "3. Menguji **Enhanced Face Features** (blink, smile, emotion detection)\n",
    "4. Menganalisis **MediaPipe Holistic** (face + pose + hands combined)\n",
    "5. Evaluasi performa dan akurasi pada berbagai kondisi\n",
    "6. Perbandingan dengan baseline head gesture system Acit\n",
    "7. Rekomendasi pengembangan \"beyond head tracking\"\n",
    "\n",
    "## 📋 Baseline Project Analysis\n",
    "**Current System (by Acit):**\n",
    "- **Technology:** MediaPipe Face Mesh\n",
    "- **Function:** Head tilt detection untuk PowerPoint control\n",
    "- **Gestures:** \n",
    "  - Tilt Right (15°): Next slide\n",
    "  - Tilt Left (15°): Previous slide  \n",
    "  - Triple Tilt (20°): Close presentation\n",
    "- **Implementation:** 468 facial landmarks, head pose calculation based on eye line angle"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📚 1. Setup dan Import Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages\n",
    "!pip install mediapipe opencv-python matplotlib seaborn pandas numpy plotly\n",
    "\n",
    "import cv2\n",
    "import mediapipe as mp\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pandas as pd\n",
    "import time\n",
    "import math\n",
    "import plotly.graph_objects as go\n",
    "import plotly.express as px\n",
    "from plotly.subplots import make_subplots\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set plotting style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"✅ All libraries imported successfully!\")\n",
    "print(f\"MediaPipe version: {mp.__version__}\")\n",
    "print(f\"OpenCV version: {cv2.__version__}\")\n",
    "print(\"\\n🎯 Project Context: Eksplorasi MediaPipe BEYOND current Head Gesture system\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 2. Baseline Analysis - Current Head Gesture System (Acit's Implementation)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize MediaPipe Face Mesh (replicating Acit's baseline system)\n",
    "mp_face_mesh = mp.solutions.face_mesh\n",
    "mp_drawing = mp.solutions.drawing_utils\n",
    "mp_drawing_styles = mp.solutions.drawing_styles\n",
    "\n",
    "# Performance tracking\n",
    "performance_data = {\n",
    "    'method': [],\n",
    "    'fps': [],\n",
    "    'detection_confidence': [],\n",
    "    'processing_time_ms': [],\n",
    "    'landmarks_count': [],\n",
    "    'accuracy_rate': [],\n",
    "    'use_case': []\n",
    "}\n",
    "\n",
    "def calculate_head_pose_acit_style(landmarks, image_size):\n",
    "    \"\"\"Replicate Acit's head pose calculation method\"\"\"\n",
    "    # Key facial landmarks for head pose estimation (same as Acit's code)\n",
    "    nose_tip = landmarks[1]\n",
    "    chin = landmarks[18]\n",
    "    left_eye_corner = landmarks[33]\n",
    "    right_eye_corner = landmarks[263]\n",
    "    \n",
    "    # Convert normalized coordinates to pixel coordinates\n",
    "    h, w = image_size\n",
    "    nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))\n",
    "    chin = (int(chin.x * w), int(chin.y * h))\n",
    "    left_eye = (int(left_eye_corner.x * w), int(left_eye_corner.y * h))\n",
    "    right_eye = (int(right_eye_corner.x * w), int(right_eye_corner.y * h))\n",
    "    \n",
    "    # Calculate head tilt (roll) - based on eye line angle (Acit's method)\n",
    "    eye_center_x = (left_eye[0] + right_eye[0]) / 2\n",
    "    eye_center_y = (left_eye[1] + right_eye[1]) / 2\n",
    "    \n",
    "    # Calculate angle of eye line\n",
    "    dx = right_eye[0] - left_eye[0]\n",
    "    dy = right_eye[1] - left_eye[1]\n",
    "    roll_angle = math.degrees(math.atan2(dy, dx))\n",
    "    \n",
    "    return {\n",
    "        'roll': roll_angle,\n",
    "        'nose_tip': nose_tip,\n",
    "        'chin': chin,\n",
    "        'left_eye': left_eye,\n",
    "        'right_eye': right_eye,\n",
    "        'eye_center': (int(eye_center_x), int(eye_center_y))\n",
    "    }\n",
    "\n",
    "def detect_head_gestures_acit_style(head_pose):\n",
    "    \"\"\"Detect head gestures using Acit's thresholds\"\"\"\n",
    "    roll = head_pose['roll']\n",
    "    \n",
    "    # Acit's thresholds\n",
    "    tilt_threshold = 15  # degrees for navigation\n",
    "    triple_tilt_threshold = 20  # degrees for exit\n",
    "    \n",
    "    if roll > tilt_threshold:\n",
    "        return \"tilt_right\", roll\n",
    "    elif roll < -tilt_threshold:\n",
    "        return \"tilt_left\", roll\n",
    "    elif abs(roll) > triple_tilt_threshold:\n",
    "        return \"triple_tilt_candidate\", roll\n",
    "    else:\n",
    "        return None, roll\n",
    "\n",
    "def test_baseline_head_gesture_system():\n",
    "    \"\"\"Test baseline head gesture system (Acit's implementation analysis)\"\"\"\n",
    "    cap = cv2.VideoCapture(0)\n",
    "    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Same as Acit's setting\n",
    "    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)\n",
    "    \n",
    "    with mp_face_mesh.FaceMesh(\n",
    "        max_num_faces=1,\n",
    "        refine_landmarks=True,\n",
    "        min_detection_confidence=0.5,\n",
    "        min_tracking_confidence=0.5\n",
    "    ) as face_mesh:\n",
    "        \n",
    "        frame_count = 0\n",
    "        start_time = time.time()\n",
    "        fps_list = []\n",
    "        processing_times = []\n",
    "        gesture_accuracy_data = []\n",
    "        head_detected_frames = 0\n",
    "        \n",
    "        print(\"🎥 Testing Baseline Head Gesture System (Acit's Method)\")\n",
    "        print(\"Silakan lakukan gesture: tilt kanan, tilt kiri, netral\")\n",
    "        print(\"Press 'q' to stop testing\")\n",
    "        \n",
    "        while frame_count < 100:  # Test for 100 frames\n",
    "            ret, frame = cap.read()\n",
    "            if not ret:\n",
    "                break\n",
    "                \n",
    "            # Flip frame horizontally (like Acit's app)\n",
    "            frame = cv2.flip(frame, 1)\n",
    "            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "            \n",
    "            # Process frame\n",
    "            process_start = time.time()\n",
    "            results = face_mesh.process(rgb_frame)\n",
    "            process_time = (time.time() - process_start) * 1000\n",
    "            processing_times.append(process_time)\n",
    "            \n",
    "            # Draw landmarks and analyze if face detected\n",
    "            if results.multi_face_landmarks:\n",
    "                head_detected_frames += 1\n",
    "                \n",
    "                for face_landmarks in results.multi_face_landmarks:\n",
    "                    # Draw face mesh contours (like Acit's visualization)\n",
    "                    mp_drawing.draw_landmarks(\n",
    "                        frame,\n",
    "                        face_landmarks,\n",
    "                        mp_face_mesh.FACEMESH_CONTOURS,\n",
    "                        landmark_drawing_spec=None,\n",
    "                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)\n",
    "                    )\n",
    "                    \n",
    "                    # Calculate head pose using Acit's method\n",
    "                    head_pose = calculate_head_pose_acit_style(face_landmarks.landmark, frame.shape[:2])\n",
    "                    \n",
    "                    # Draw head pose indicators (like Acit's app)\n",
    "                    nose_tip = head_pose['nose_tip']\n",
    "                    eye_center = head_pose['eye_center']\n",
    "                    \n",
    "                    cv2.circle(frame, nose_tip, 5, (0, 0, 255), -1)\n",
    "                    cv2.circle(frame, eye_center, 3, (255, 0, 0), -1)\n",
    "                    cv2.line(frame, eye_center, nose_tip, (255, 255, 0), 2)\n",
    "                    \n",
    "                    # Detect gestures\n",
    "                    gesture, roll_angle = detect_head_gestures_acit_style(head_pose)\n",
    "                    \n",
    "                    # Display head tilt angle (like Acit's app)\n",
    "                    cv2.putText(frame, f\"Head Tilt: {roll_angle:.1f}°\", (10, 30), \n",
    "                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)\n",
    "                    \n",
    "                    # Display detected gesture\n",
    "                    if gesture:\n",
    "                        gesture_text = {\n",
    "                            \"tilt_right\": \"TILT RIGHT - NEXT SLIDE\",\n",
    "                            \"tilt_left\": \"TILT LEFT - PREVIOUS SLIDE\",\n",
    "                            \"triple_tilt_candidate\": \"STRONG TILT - TRIPLE TILT CANDIDATE\"\n",
    "                        }\n",
    "                        cv2.putText(frame, gesture_text.get(gesture, gesture), (10, 70), \n",
    "                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)\n",
    "                    \n",
    "                    # Store gesture data for accuracy analysis\n",
    "                    gesture_accuracy_data.append({\n",
    "                        'frame': frame_count,\n",
    "                        'roll_angle': roll_angle,\n",
    "                        'gesture_detected': gesture,\n",
    "                        'processing_time': process_time\n",
    "                    })\n",
    "            \n",
    "            # Calculate FPS\n",
    "            frame_count += 1\n",
    "            if frame_count % 30 == 0:\n",
    "                elapsed_time = time.time() - start_time\n",
    "                fps = 30 / elapsed_time\n",
    "                fps_list.append(fps)\n",
    "                start_time = time.time()\n",
    "            \n",
    "            # Display frame counter and detection status\n",
    "            cv2.putText(frame, f'Frame: {frame_count}/100', (10, frame.shape[0] - 60), \n",
    "                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)\n",
    "            cv2.putText(frame, f'Head Detection: {head_detected_frames}/{frame_count}', \n",
    "                       (10, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, \n",
    "                       (0, 255, 0) if results.multi_face_landmarks else (0, 0, 255), 2)\n",
    "            \n",
    "            # Show instructions\n",
    "            instructions = [\n",
    "                \"Tilt Right: Next slide (15°+)\",\n",
    "                \"Tilt Left: Previous slide (15°+)\", \n",
    "                \"Strong Tilt: Triple tilt candidate (20°+)\",\n",
    "                \"Keep head visible in frame\"\n",
    "            ]\n",
    "            \n",
    "            for i, instruction in enumerate(instructions):\n",
    "                cv2.putText(frame, instruction, (10, 100 + i * 25), \n",
    "                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)\n",
    "            \n",
    "            cv2.imshow('Baseline Head Gesture Analysis (Acit\\'s Method)', frame)\n",
    "            \n",
    "            if cv2.waitKey(1) & 0xFF == ord('q'):\n",
    "                break\n",
    "    \n",
    "    cap.release()\n",
    "    cv2.destroyAllWindows()\n",
    "    \n",
    "    # Calculate performance metrics\n",
    "    avg_fps = np.mean(fps_list) if fps_list else 0\n",
    "    avg_processing_time = np.mean(processing_times)\n",
    "    detection_rate = head_detected_frames / frame_count if frame_count > 0 else 0\n",
    "    \n",
    "    # Analyze gesture accuracy\n",
    "    gesture_df = pd.DataFrame(gesture_accuracy_data)\n",
    "    successful_gestures = len(gesture_df[gesture_df['gesture_detected'].notna()])\n",
    "    gesture_accuracy = successful_gestures / len(gesture_df) if len(gesture_df) > 0 else 0\n",
    "    \n",
    "    # Store performance data\n",
    "    performance_data['method'].append('Head Gesture (Baseline - Acit\\'s Method)')\n",
    "    performance_data['fps'].append(avg_fps)\n",
    "    performance_data['processing_time_ms'].append(avg_processing_time)\n",
    "    performance_data['landmarks_count'].append(468)  # Face mesh landmarks\n",
    "    performance_data['detection_confidence'].append(detection_rate)\n",
    "    performance_data['accuracy_rate'].append(gesture_accuracy)\n",
    "    performance_data['use_case'].append('PowerPoint Control')\n",
    "    \n",
    "    print(f\"\\n✅ Baseline Head Gesture Analysis completed!\")\n",
    "    print(f\"Average FPS: {avg_fps:.2f}\")\n",
    "    print(f\"Average Processing Time: {avg_processing_time:.2f}ms\")\n",
    "    print(f\"Head Detection Rate: {detection_rate:.2%}\")\n",
    "    print(f\"Gesture Detection Accuracy: {gesture_accuracy:.2%}\")\n",
    "    print(f\"Total Gestures Detected: {successful_gestures}/{len(gesture_df)}\")\n",
    "    \n",
    "    return {\n",
    "        'avg_fps': avg_fps,\n",
    "        'avg_processing_time': avg_processing_time,\n",
    "        'detection_rate': detection_rate,\n",
    "        'gesture_accuracy': gesture_accuracy,\n",
    "        'landmarks_count': 468,\n",
    "        'gesture_data': gesture_df\n",
    "    }\n",
    "\n",
    "# Run baseline test\n",
    "baseline_results = test_baseline_head_gesture_system()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🏃‍♂️ 3. MediaPipe Pose - Beyond Head: Full Body Tracking"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize MediaPipe Pose\n",
    "mp_pose = mp.solutions.pose\n",
    "\n",
    "def test_pose_body_gestures():\n",
    "    \"\"\"Test MediaPipe Pose for full body gesture control beyond head\"\"\"\n",
    "    cap = cv2.VideoCapture(0)\n",
    "    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)\n",
    "    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)\n",
    "    \n",
    "    with mp_pose.Pose(\n",
    "        min_detection_confidence=0.5,\n",
    "        min_tracking_confidence=0.5,\n",
    "        model_complexity=1  # 0=Lite, 1=Full, 2=Heavy\n",
    "    ) as pose:\n",
    "        \n",
    "        frame_count = 0\n",
    "        start_time = time.time()\n",
    "        fps_list = []\n",
    "        processing_times = []\n",
    "        pose_detected_frames = 0\n",
    "        gesture_data = []\n",
    "        \n",
    "        print(\"🏃‍♂️ Testing Body Pose Gestures (Beyond Head Tracking)\")\n",
    "        print(\"Try: Raise hand, point left/right, arms crossed, standing/sitting\")\n",
    "        print(\"Press 'q' to stop\")\n",
    "        \n",
    "        while frame_count < 100:\n",
    "            ret, frame = cap.read()\n",
    "            if not ret:\n",
    "                break\n",
    "                \n",
    "            frame = cv2.flip(frame, 1)\n",
    "            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "            \n",
    "            # Process frame\n",
    "            process_start = time.time()\n",
    "            results = pose.process(rgb_frame)\n",
    "            process_time = (time.time() - process_start) * 1000\n",
    "            processing_times.append(process_time)\n",
    "            \n",
    "            # Draw pose landmarks\n",
    "            if results.pose_landmarks:\n",
    "                pose_detected_frames += 1\n",
    "                mp_drawing.draw_landmarks(\n",
    "                    frame,\n",
    "                    results.pose_landmarks,\n",
    "                    mp_pose.POSE_CONNECTIONS,\n",
    "                    mp_drawing_styles.get_default_pose_landmarks_style()\n",
    "                )\n",
    "                \n",
    "                # Extract key body pose features\n",
    "                landmarks = results.pose_landmarks.landmark\n",
    "                \n",
    "                # Body measurements and gesture detection\n",
    "                left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]\n",
    "                right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]\n",
    "                left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]\n",
    "                right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]\n",
    "                left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]\n",
    "                right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]\n",
    "                nose = landmarks[mp_pose.PoseLandmark.NOSE]\n",
    "                \n",
    "                # Calculate body metrics\n",
    "                shoulder_width = abs(left_shoulder.x - right_shoulder.x)\n",
    "                shoulder_y_avg = (left_shoulder.y + right_shoulder.y) / 2\n",
    "                \n",
    "                # Hand position analysis (beyond head gestures)\n",
    "                left_hand_raised = left_wrist.y < left_shoulder.y - 0.1\n",
    "                right_hand_raised = right_wrist.y < right_shoulder.y - 0.1\n",
    "                both_hands_raised = left_hand_raised and right_hand_raised\n",
    "                \n",
    "                # Pointing gestures\n",
    "                pointing_left = right_wrist.x < right_shoulder.x - 0.2 and right_wrist.y < right_shoulder.y\n",
    "                pointing_right = left_wrist.x > left_shoulder.x + 0.2 and left_wrist.y < left_shoulder.y\n",
    "                \n",
    "                # Arms crossed detection\n",
    "                arms_crossed = (left_wrist.x > right_shoulder.x - 0.1 and \n",
    "                               right_wrist.x < left_shoulder.x + 0.1 and\n",
    "                               abs(left_wrist.y - right_wrist.y) < 0.15)\n",
    "                \n",
    "                # Body leaning detection\n",
    "                body_lean_left = (left_shoulder.y > right_shoulder.y + 0.05)\n",
    "                body_lean_right = (right_shoulder.y > left_shoulder.y + 0.05)\n",
    "                \n",
    "                # Gesture classification for presentation control\n",
    "                detected_gestures = []\n",
    "                \n",
    "                if both_hands_raised:\n",
    "                    detected_gestures.append(\"BOTH_HANDS_UP\")\n",
    "                elif left_hand_raised and not right_hand_raised:\n",
    "                    detected_gestures.append(\"LEFT_HAND_UP\")\n",
    "                elif right_hand_raised and not left_hand_raised:\n",
    "                    detected_gestures.append(\"RIGHT_HAND_UP\")\n",
    "                \n",
    "                if pointing_left:\n",
    "                    detected_gestures.append(\"POINTING_LEFT\")\n",
    "                elif pointing_right:\n",
    "                    detected_gestures.append(\"POINTING_RIGHT\")\n",
    "                \n",
    "                if arms_crossed:\n",
    "                    detected_gestures.append(\"ARMS_CROSSED\")\n",
    "                \n",
    "                if body_lean_left:\n",
    "                    detected_gestures.append(\"LEAN_LEFT\")\n",
    "                elif body_lean_right:\n",
    "                    detected_gestures.append(\"LEAN_RIGHT\")\n",
    "                \n",
    "                # Display body metrics\n",
    "                cv2.putText(frame, f'Shoulder Width: {shoulder_width:.3f}', \n",
    "                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)\n",
    "                \n",
    "                # Display detected gestures\n",
    "                if detected_gestures:\n",
    "                    gesture_text = \" | \".join(detected_gestures)\n",
    "                    cv2.putText(frame, f'Gestures: {gesture_text}', \n",
    "                               (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)\n",
    "                \n",
    "                # Potential presentation control mapping\n",
    "                control_suggestion = \"\"\n",
    "                if \"RIGHT_HAND_UP\" in detected_gestures:\n",
    "                    control_suggestion = \"→ NEXT SLIDE\"\n",
    "                elif \"LEFT_HAND_UP\" in detected_gestures:\n",
    "                    control_suggestion = \"← PREVIOUS SLIDE\"\n",
    "                elif \"BOTH_HANDS_UP\" in detected_gestures:\n",
    "                    control_suggestion = \"↑ START/STOP PRESENTATION\"\n",
    "                elif \"ARMS_CROSSED\" in detected_gestures:\n",
    "                    control_suggestion = \"✕ EXIT PRESENTATION\"\n",
    "                elif \"POINTING_LEFT\" in detected_gestures:\n",
    "                    control_suggestion = \"← JUMP TO BEGINNING\"\n",
    "                elif \"POINTING_RIGHT\" in detected_gestures:\n",
    "                    control_suggestion = \"→ JUMP TO END\"\n",
    "                \n",
    "                if control_suggestion:\n",
    "                    cv2.putText(frame, f'Control: {control_suggestion}', \n",
    "                               (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)\n",
    "                \n",
    "                # Store gesture data\n",
    "                gesture_data.append({\n",
    "                    'frame': frame_count,\n",
    "                    'shoulder_width': shoulder_width,\n",
    "                    'left_hand_raised': left_hand_raised,\n",
    "                    'right_hand_raised': right_hand_raised,\n",
    "                    'gestures': detected_gestures,\n",
    "                    'control_suggestion': control_suggestion\n",
    "                })\n",
    "            \n",
    "            # Calculate FPS\n",
    "            frame_count += 1\n",
    "            if frame_count % 30 == 0:\n",
    "                elapsed_time = time.time() - start_time\n",
    "                fps = 30 / elapsed_time\n",
    "                fps_list.append(fps)\n",
    "                start_time = time.time()\n",
    "            \n",
    "            # Display info\n",
    "            cv2.putText(frame, f'Frame: {frame_count}/100', (10, frame.shape[0] - 60), \n",
    "                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)\n",
    "            cv2.putText(frame, f'Pose Detection: {pose_detected_frames}/{frame_count}', \n",
    "                       (10, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, \n",
    "                       (0, 255, 0) if results.pose_landmarks else (0, 0, 255), 2)\n",
    "            \n",
    "            cv2.imshow('Body Pose Gestures (Beyond Head)', frame)\n",
    "            \n",
    "            if cv2.waitKey(1) & 0xFF == ord('q'):\n",
    "                break\n",
    "    \n",
    "    cap.release()\n",
    "    cv2.destroyAllWindows()\n",
    "    \n",
    "    # Store performance data\n",
    "    avg_fps = np.mean(fps_list) if fps_list else 0\n",
    "    avg_processing_time = np.mean(processing_times)\n",
    "    detection_rate = pose_detected_frames / frame_count if frame_count > 0 else 0\n",
    "    \n",
    "    # Analyze gesture variety\n",
    "    gesture_df = pd.DataFrame(gesture_data)\n",
    "    unique_gestures = set()\n",
    "    for gestures_list in gesture_df['gestures']:\n",
    "        unique_gestures.update(gestures_list)\n",
    "    gesture_variety = len(unique_gestures)\n",
    "    \n",
    "    performance_data['method'].append('Body Pose Gestures')\n",
    "    performance_data['fps'].append(avg_fps)\n",
    "    performance_data['processing_time_ms'].append(avg_processing_time)\n",
    "    performance_data['landmarks_count'].append(33)  # Pose landmarks\n",
    "    performance_data['detection_confidence'].append(detection_rate)\n",
    "    performance_data['accuracy_rate'].append(gesture_variety / 10)  # Normalized variety score\n",
    "    performance_data['use_case'].append('Body Control & Fitness')\n",
    "    \n",
    "    print(f\"\\n✅ Body Pose Gesture Analysis completed!\")\n",
    "    print(f\"Average FPS: {avg_fps:.2f}\")\n",
    "    print(f\"Average Processing Time: {avg_processing_time:.2f}ms\")\n",
    "    print(f\"Pose Detection Rate: {detection_rate:.2%}\")\n",
    "    print(f\"Unique Gestures Detected: {gesture_variety}\")\n",
    "    print(f\"Detected gesture types: {list(unique_gestures)}\")\n",
    "    \n",
    "    return {\n",
    "        'avg_fps': avg_fps,\n",
    "        'avg_processing_time': avg_processing_time,\n",
    "        'detection_rate': detection_rate,\n",
    "        'gesture_variety': gesture_variety,\n",
    "        'landmarks_count': 33,\n",
    "        'gesture_data': gesture_df\n",
    "    }\n",
    "\n",
    "# Run pose detection test\n",
    "pose_results = test_pose_body_gestures()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎭 4. Enhanced Face Features - Beyond Basic Head Tilt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def test_enhanced_face_features_beyond_head():\n",
    "    \"\"\"Test advanced face features beyond basic head tilt detection\"\"\"\n",
    "    cap = cv2.VideoCapture(0)\n",
    "    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)\n",
    "    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)\n",
    "    \n",
    "    with mp_face_mesh.FaceMesh(\n",
    "        max_num_faces=2,  # Multiple faces\n",
    "        refine_landmarks=True,\n",
    "        min_detection_confidence=0.7,\n",
    "        min_tracking_confidence=0.7\n",
    "    ) as face_mesh:\n",
    "        \n",
    "        frame_count = 0\n",
    "        start_time = time.time()\n",
    "        fps_list = []\n",
    "        processing_times = []\n",
    "        facial_expression_data = []\n",
    "        \n",
    "        print(\"🎭 Testing Enhanced Face Features (Beyond Head Tilt)\")\n",
    "        print(\"Try: Blink, smile, open mouth, raise eyebrows, look around\")\n",
    "        print(\"Press 'q' to stop\")\n",
    "        \n",
    "        while frame_count < 100:\n",
    "            ret, frame = cap.read()\n",
    "            if not ret:\n",
    "                break\n",
    "                \n",
    "            frame = cv2.flip(frame, 1)\n",
    "            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "            h, w, _ = frame.shape\n",
    "            \n",
    "            # Process frame\n",
    "            process_start = time.time()\n",
    "            results = face_mesh.process(rgb_frame)\n",
    "            process_time = (time.time() - process_start) * 1000\n",
    "            processing_times.append(process_time)\n",
    "            \n",
    "            if results.multi_face_landmarks:\n",
    "                for idx, face_landmarks in enumerate(results.multi_face_landmarks):\n",
    "                    # Draw refined landmarks\n",
    "                    mp_drawing.draw_landmarks(\n",
    "                        frame,\n",
    "                        face_landmarks,\n",
    "                        mp_face_mesh.FACEMESH_TESSELATION,\n",
    "                        None,\n",
    "                        mp_drawing_styles.get_default_face_mesh_tesselation_style()\n",
    "                    )\n",
    "                    \n",
    "                    # Extract advanced facial features (beyond head tilt)\n",
    "                    landmarks = face_landmarks.landmark\n",
    "                    \n",
    "                    # Eye analysis (blink detection)\n",
    "                    left_eye_top = landmarks[159]\n",
    "                    left_eye_bottom = landmarks[145]\n",
    "                    right_eye_top = landmarks[386]\n",
    "                    right_eye_bottom = landmarks[374]\n",
    "                    \n",
    "                    left_eye_height = abs(left_eye_top.y - left_eye_bottom.y)\n",
    "                    right_eye_height = abs(right_eye_top.y - right_eye_bottom.y)\n",
    "                    avg_eye_height = (left_eye_height + right_eye_height) / 2\n",
    "                    \n",
    "                    # Eyebrow analysis (surprise detection)\n",
    "                    left_eyebrow = landmarks[70]\n",
    "                    right_eyebrow = landmarks[300]\n",
    "                    eyebrow_height = (left_eyebrow.y + right_eyebrow.y) / 2\n",
    "                    \n",
    "                    # Mouth analysis (smile, open mouth detection)\n",
    "                    mouth_left = landmarks[61]\n",
    "                    mouth_right = landmarks[291]\n",
    "                    mouth_top = landmarks[13]\n",
    "                    mouth_bottom = landmarks[14]\n",
    "                    \n",
    "                    mouth_width = abs(mouth_left.x - mouth_right.x)\n",
    "                    mouth_height = abs(mouth_top.y - mouth_bottom.y)\n",
    "                    mouth_ratio = mouth_width / mouth_height if mouth_height > 0 else 0\n",
    "                    \n",
    "                    # Eye gaze direction (beyond head tilt)\n",
    "                    left_eye_center = landmarks[468]\n",
    "                    right_eye_center = landmarks[473]\n",
    "                    nose_tip = landmarks[1]\n",
    "                    \n",
    "                    # Calculate gaze direction\n",
    "                    eye_center_x = (left_eye_center.x + right_eye_center.x) / 2\n",
    "                    gaze_offset = nose_tip.x - eye_center_x\n",
    "                    \n",
    "                    # Advanced expression detection\n",
    "                    is_blinking = avg_eye_height < 0.008\n",
    "                    is_smiling = mouth_ratio > 3.2\n",
    "                    is_mouth_open = mouth_height > 0.02\n",
    "                    is_surprised = eyebrow_height < 0.3  # Higher eyebrows\n",
    "                    gaze_direction = \"CENTER\"\n",
    "                    \n",
    "                    if gaze_offset > 0.02:\n",
    "                        gaze_direction = \"RIGHT\"\n",
    "                    elif gaze_offset < -0.02:\n",
    "                        gaze_direction = \"LEFT\"\n",
    "                    \n",
    "                    # Advanced gesture classification for presentation control\n",
    "                    advanced_gestures = []\n",
    "                    control_commands = []\n",
    "                    \n",
    "                    if is_blinking:\n",
    "                        advanced_gestures.append(\"BLINK\")\n",
    "                        control_commands.append(\"→ CLICK/SELECT\")\n",
    "                    \n",
    "                    if is_smiling:\n",
    "                        advanced_gestures.append(\"SMILE\")\n",
    "                        control_commands.append(\"→ POSITIVE FEEDBACK\")\n",
    "                    \n",
    "                    if is_mouth_open:\n",
    "                        advanced_gestures.append(\"MOUTH_OPEN\")\n",
    "                        control_commands.append(\"→ VOICE COMMAND READY\")\n",
    "                    \n",
    "                    if is_surprised:\n",
    "                        advanced_gestures.append(\"EYEBROWS_UP\")\n",
    "                        control_commands.append(\"→ ATTENTION/HIGHLIGHT\")\n",
    "                    \n",
    "                    if gaze_direction != \"CENTER\":\n",
    "                        advanced_gestures.append(f\"GAZE_{gaze_direction}\")\n",
    "                        control_commands.append(f\"→ LOOK {gaze_direction}\")\n",
    "                    \n",
    "                    # Display analysis results\n",
    "                    y_offset = 30 + (idx * 200)\n",
    "                    \n",
    "                    cv2.putText(frame, f'Face {idx+1} Advanced Features:', (10, y_offset), \n",
    "                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)\n",
    "                    \n",
    "                    cv2.putText(frame, f'Eye: {avg_eye_height:.4f} | Mouth: {mouth_ratio:.2f}', \n",
    "                               (10, y_offset + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)\n",
    "                    \n",
    "                    cv2.putText(frame, f'Gaze: {gaze_direction} | Eyebrow: {eyebrow_height:.3f}', \n",
    "                               (10, y_offset + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)\n",
    "                    \n",
    "                    if advanced_gestures:\n",
    "                        gesture_text = \" | \".join(advanced_gestures)\n",
    "                        cv2.putText(frame, f'Expressions: {gesture_text}', \n",
    "                                   (10, y_offset + 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)\n",
    "                    \n",
    "                    if control_commands:\n",
    "                        command_text = \" \".join(control_commands[:2])  # Show first 2 commands\n",
    "                        cv2.putText(frame, f'Controls: {command_text}', \n",
    "                                   (10, y_offset + 95), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)\n",
    "                    \n",
    "                    # Store facial expression data\n",
    "                    facial_expression_data.append({\n",
    "                        'frame': frame_count,\n",
    "                        'face_id': idx,\n",
    "                        'eye_height': avg_eye_height,\n",
    "                        'mouth_ratio': mouth_ratio,\n",
    "                        'mouth_height': mouth_height,\n",
    "                        'eyebrow_height': eyebrow_height,\n",
    "                        'gaze_direction': gaze_direction,\n",
    "                        'is_blinking': is_blinking,\n",
    "                        'is_smiling': is_smiling,\n",
    "                        'is_mouth_open': is_mouth_open,\n",
    "                        'is_surprised': is_surprised,\n",
    "                        'advanced_gestures': advanced_gestures,\n",
    "                        'control_commands': control_commands\n",
    "                    })\n",
    "            \n",
    "            # Calculate FPS\n",
    "            frame_count += 1\n",
    "            if frame_count % 30 == 0:\n",
    "                elapsed_time = time.time() - start_time\n",
    "                fps = 30 / elapsed_time\n",
    "                fps_list.append(fps)\n",
    "                start_time = time.time()\n",
    "            \n",
    "            cv2.putText(frame, f'Frame: {frame_count}/100', (10, frame.shape[0] - 60), \n",
    "                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)\n",
    "            cv2.putText(frame, f'Faces: {len(results.multi_face_landmarks) if results.multi_face_landmarks else 0}', \n",
    "                       (10, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)\n",
    "            \n",
    "            cv2.imshow('Enhanced Face Features (Beyond Head Tilt)', frame)\n",
    "            \n",
    "            if cv2.waitKey(1) & 0xFF == ord('q'):\n",
    "                break\n",
    "    \n",
    "    cap.release()\n",
    "    cv2.destroyAllWindows()\n",
    "    \n",
    "    # Calculate performance metrics\n",
    "    avg_fps = np.mean(fps_list) if fps_list else 0\n",
    "    avg_processing_time = np.mean(processing_times)\n",
    "    \n",
    "    # Analyze expression variety and accuracy\n",
    "    expression_df = pd.DataFrame(facial_expression_data)\n",
    "    unique_expressions = set()\n",
    "    for expr_list in expression_df['advanced_gestures']:\n",
    "        unique_expressions.update(expr_list)\n",
    "    expression_variety = len(unique_expressions)\n",
    "    \n",
    "    # Calculate expression detection accuracy\n",
    "    total_detections = len(expression_df[expression_df['advanced_gestures'].apply(len) > 0])\n",
    "    expression_accuracy = total_detections / len(expression_df) if len(expression_df) > 0 else 0\n",
    "    \n",
    "    performance_data['method'].append('Enhanced Face Features')\n",
    "    performance_data['fps'].append(avg_fps)\n",
    "    performance_data['processing_time_ms'].append(avg_processing_time)\n",
    "    performance_data['landmarks_count'].append(468)\n",
    "    performance_data['detection_confidence'].append(0.7)\n",
    "    performance_data['accuracy_rate'].append(expression_accuracy)\n",
    "    performance_data['use_case'].append('Emotion AI & Accessibility')\n",
    "    \n",
    "    print(f\"\\n✅ Enhanced Face Features Analysis completed!\")\n",
    "    print(f\"Average FPS: {avg_fps:.2f}\")\n",
    "    print(f\"Average Processing Time: {avg_processing_time:.2f}ms\")\n",
    "    print(f\"Expression Detection Accuracy: {expression_accuracy:.2%}\")\n",
    "    print(f\"Unique Expressions Detected: {expression_variety}\")\n",
    "    print(f\"Expression types: {list(unique_expressions)}\")\n",
    "    \n",
    "    return {\n",
    "        'avg_fps': avg_fps,\n",
    "        'avg_processing_time': avg_processing_time,\n",
    "        'expression_accuracy': expression_accuracy,\n",
    "        'expression_variety': expression_variety,\n",
    "        'landmarks_count': 468,\n",
    "        'expression_data': expression_df\n",
    "    }\n",
    "\n",
    "# Run enhanced face features test\n",
    "enhanced_face_results = test_enhanced_face_features_beyond_head()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
```

SyntaxError: invalid syntax (233752286.py, line 1)